<a href="https://colab.research.google.com/github/thomouvic/CSC502/blob/main/pyspark_matrix_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix Multiplication with Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop2.tgz
!tar xf spark-3.3.2-bin-hadoop2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop2"

import findspark
findspark.init("spark-3.3.2-bin-hadoop2")# SPARK_HOME

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

In [6]:
M1 = M.map(lambda x: (x[1], ('M',x[0],x[2])))	# x[1] is j
N1 = N.map(lambda x: (x[0], ('N',x[1],x[2])))	# x[0] is j

print(M1.collect())
print(N1.collect())

[(1, ('M', 1, 1)), (2, ('M', 1, 3)), (1, ('M', 2, 5)), (2, ('M', 2, 7))]
[(1, ('N', 1, 2)), (1, ('N', 2, 4)), (2, ('N', 1, 6)), (2, ('N', 2, 8))]


In [3]:
# M = [1  3
#      5  7]

# N = [2  4
#      6  8]

M = sc.parallelize([
     (1,1,1), (1,2,3),
     (2,1,5), (2,2,7)])

N = sc.parallelize([
     (1,1,2), (1,2,4),
     (2,1,6), (2,2,8)])

In [20]:
MN = M1.join(N1)	# join on key

print(MN.take(3))

[(1, (('M', 1, 1), ('N', 1, 2))), (1, (('M', 1, 1), ('N', 2, 4))), (1, (('M', 2, 5), ('N', 1, 2)))]


In [17]:
def f(li):
    # li is of the form (('M', i, m), ('N', k, n))
    i = li[0][1]; k = li[1][1]
    m = li[0][2]; n = li[1][2]
    return ((i, k), m*n)

In [18]:
MNR1 = MN.map(lambda x: f(x[1]))

print(MNR1.collect())

[((1, 1), 2), ((1, 2), 4), ((2, 1), 10), ((2, 2), 20), ((1, 1), 18), ((1, 2), 24), ((2, 1), 42), ((2, 2), 56)]


In [19]:
Final = MNR1.reduceByKey(lambda a,b: a+b)
print(Final.collect())

[((1, 1), 20), ((1, 2), 28), ((2, 2), 76), ((2, 1), 52)]
